In [30]:
from sqlalchemy import create_engine ,Integer, String, Date,VARCHAR,text
import pandas as pd

In [31]:

pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"C:\Users\Note reserva\Desktop\Curso Engenharia Dados\Arquivos\01. Postgree\Origem de dados\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )


In [32]:
df.head()

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT,PA-34-200


In [33]:
df.shape[0]

13011

In [34]:
from datetime import datetime, timedelta

#Método 1
data_atual = datetime.now()# Data Atual
data_ha_3_meses = data_atual - timedelta(days=3*30) # Subtrai 3 meses usando timedelta 
data_ha_3_meses_apenas_data = data_ha_3_meses.date() # traz somente a data , exclui hora minuto segundo etc

#Obs:Não leva em consideração a variação nos números de dias em cada mês, resultando em uma aproximação.
print(data_ha_3_meses_apenas_data)

2024-11-04


In [35]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

#Método 2
data_atual2 = datetime.now()
d_menos3M = data_atual - relativedelta(months=3) # Subtrai 3 meses usando relativedelta
dt_d_menos3M = d_menos3M.date()

#Obs:Leva em consideração a variação nos números de dias em cada mês, fornecendo um resultado mais preciso
print(dt_d_menos3M)


2024-11-02


In [36]:
df.dtypes

Numero_da_Ocorrencia            int64
Classificacao_da_Ocorrencia    object
Data_da_Ocorrencia             object
Municipio                      object
UF                             object
Regiao                         object
Nome_do_Fabricante             object
Modelo                         object
dtype: object

In [37]:
df.head()

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT,PA-34-200


In [38]:
# Converter tudo para Pandas to_datetime para fazer o filtro na base 
df['Data_da_Ocorrencia']    = pd.to_datetime(df['Data_da_Ocorrencia'])
data_ha_3_meses_apenas_data = pd.to_datetime(data_ha_3_meses_apenas_data)
dt_d_menos3M                = pd.to_datetime(dt_d_menos3M)

In [39]:
df.shape[0]

13011

In [40]:
df = df[df['Data_da_Ocorrencia'] >= dt_d_menos3M]

In [41]:
df.head()

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo


In [42]:
dt_d_menos3M_ajustado=dt_d_menos3M.date()

In [43]:
#conexao com banco de dados
dbname   = 'python'
user     = 'postgres'
password = '1022'
host     = 'localhost'
port     = '5432' 

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

nome_tabela = 'anac_sqlalchemy' 

# Deletar registros com base no ano atual
cursor=engine.connect() 
delete = text(f'''    delete from public.{nome_tabela}	
                      WHERE "Data_da_Ocorrencia" >=  '{dt_d_menos3M_ajustado}'
                ''')
cursor.execute(delete)
cursor.commit()

# Enviar DataFrame para o banco de dados
df.to_sql(nome_tabela, engine, index=False, if_exists='append',
                     dtype={ 
                           'Numero_da_Ocorrencia' :   Integer ,
                           'Classificacao_da_Ocorrencia': VARCHAR(50),
                           'Data_da_Ocorrencia':Date  
                           })

engine.dispose()
cursor.close()